This notebook allows collecting results across multiple pipeline runs. Concatenating across sessions and bin sizes. 
It takes CSVs, then determines the most recent one from the filename. 

There are two major sets of decoded epochs - Laps and Ripples/Replays
There are two sets of marginals for the decoded epochs - the "by epoch" and "by time bin" marginals.
The "by time bin" epochs are a larger granulation, with each epoch consisting of one or more time bin.


```python

# 2024-01-23 - 
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-fet11-01_12-58-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-03_12-3-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_17-46-44_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_19-28-0_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-58-3_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-25-50_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_16-40-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_22-24-40_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_16-53-46_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_17-29-30_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_14-26-15_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_1-22-43_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-07_16-40-19_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_21-16-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_15-55-31_time_bin_size_sweep_results.h5



# found_session_export_paths = [Path(v).resolve() for v in  ["C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# ]]

[
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-17_0210AM-kdiba_vvp01_one_2006-4-09_17-29-30-(laps_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(laps_weighted_corr_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_weighted_corr_merged_df)_tbin-0.025.csv",
]

```

In [2]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# Switch to the desired interactivity mode
plt.interactive(True)

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _helper_build_figure, plotly_pre_post_delta_scatter, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import find_csv_files, find_HDF5_files, find_most_recent_files, process_csv_file, plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult

debug_print: bool = False

TODAY_DAY_DATE: str = f"2024-03-14_GL"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier


def process_and_load_file(session_dict, df_file_name_key, loaded_dict, session_name, curr_session_t_delta, time_key):
    try:
        file_path = session_dict[df_file_name_key]
        loaded_dict[session_name] = process_csv_file(file_path, session_name, curr_session_t_delta, time_key)
    except BaseException as e:
        print(f'session {session_name} did not fully work. (error {e}. Skipping.')


# def prototype_flexible_process_and_load_file(session_dict, df_file_name_key, loaded_dict, session_name, curr_session_t_delta, time_key):
#     """ supposed to not require a bunch of manually created input variable names like:

#     final_sessions_loaded_laps_dict = {}
#     final_sessions_loaded_ripple_dict = {}
#     final_sessions_loaded_laps_time_bin_dict = {}
#     final_sessions_loaded_ripple_time_bin_dict = {}

#     and then processing them separately

#     Instead it splits the df_file_name_key like 'ripple_weighted_corr_merged_df' -> ('ripple', 'weighted_corr_merged_df')
#         and assigns to a nested dictionary
    
#     """
#     (an_epochs_source_name, a_df_variable_name) = df_file_name_key.split(sep='_', maxsplit=1) # get the first part of the variable names that indicates whether it's for "laps" or "ripple"
#     # like 'ripple_weighted_corr_merged_df' -> ('ripple', 'weighted_corr_merged_df')
#     try:
#         file_path = session_dict[df_file_name_key]
#         if an_epochs_source_name not in loaded_dict[session_name]:
#             loaded_dict[session_name][an_epochs_source_name] = {}
#         loaded_dict[session_name][an_epochs_source_name][a_df_variable_name] = process_csv_file(file_path, session_name, curr_session_t_delta, time_key)
#     except BaseException as e:
#         print(f'session {session_name} did not fully work. (error {e}. Skipping.')


def _common_cleanup_operations(a_df):
    """ post loading and concatenation across sessions dataframe cleanup """
    if a_df is None:
        return None
    ## Drop the weird 'Unnamed: 0' column:
    # Rename column 'Unnamed: 0' to 'abs_time_bin_index'
    a_df = a_df.rename(columns={'Unnamed: 0': 'abs_time_bin_index'})
    # Drop column: 'abs_time_bin_index'
    a_df = a_df.drop(columns=['abs_time_bin_index'])
    # Add additional 'epoch_idx' column for compatibility:
    if 'epoch_idx' not in a_df:
        if 'lap_idx' in a_df:
            a_df['epoch_idx'] = a_df['lap_idx']
        if 'ripple_idx' in a_df:
            a_df['epoch_idx'] = a_df['ripple_idx']
    return a_df

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
TODAY_DAY_DATE: 2024-03-14_GL


# Definitions
"_new" (as in `all_sessions_new_ripple_df`) adds simple_pearson and a few other measures

In [6]:
## Load across session t_delta CSV, which contains the t_delta for each session:

# t_delta_csv_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs\2024-01-18_GL_t_split_df.csv').resolve() # Apogee
# t_delta_csv_path = Path('/home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-01-18_GL_t_split_df.csv').resolve() # GL

# collected_outputs_directory = '/home/halechr/FastData/collected_outputs/'
# collected_outputs_directory = r'C:\Users\pho\Desktop\collected_outputs'
# collected_outputs_directory = r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs' # APOGEE
# collected_outputs_directory = '/home/halechr/cloud/turbo/Data/Output/collected_outputs' # GL

known_collected_outputs_paths = [Path(v).resolve() for v in [r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs', '/home/halechr/cloud/turbo/Data/Output/collected_outputs', '/home/halechr/FastData/collected_outputs/']]
collected_outputs_directory = find_first_extant_path(known_collected_outputs_paths)
assert collected_outputs_directory.exists(), f"collected_outputs_directory: {collected_outputs_directory} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_directory: {collected_outputs_directory}')

t_delta_csv_path = collected_outputs_directory.joinpath('2024-01-18_GL_t_split_df.csv').resolve() # GL
assert t_delta_csv_path.exists()

## Find the files:
csv_files = find_csv_files(collected_outputs_directory)
h5_files = find_HDF5_files(collected_outputs_directory)

csv_sessions = find_most_recent_files(found_session_export_paths=csv_files)
h5_sessions = find_most_recent_files(found_session_export_paths=h5_files)

## The CSV containing the session delta time:
t_delta_df = pd.read_csv(t_delta_csv_path, index_col=0) # Assuming that your CSV file has an index column
# adds `delta_aligned_t_start`, `delta_aligned_t_end` columns
t_delta_df['delta_aligned_t_start'] = t_delta_df['t_start'] - t_delta_df['t_delta']
t_delta_df['delta_aligned_t_end'] = t_delta_df['t_end'] - t_delta_df['t_delta']

# computes `earliest_delta_aligned_t_start`, latest_delta_aligned_t_end
earliest_delta_aligned_t_start: float = np.nanmin(t_delta_df['delta_aligned_t_start'])
latest_delta_aligned_t_end: float = np.nanmax(t_delta_df['delta_aligned_t_end'])
print(f'earliest_delta_aligned_t_start: {earliest_delta_aligned_t_start}, latest_delta_aligned_t_end: {latest_delta_aligned_t_end}')
t_delta_dict = t_delta_df.to_dict(orient='index')
# t_delta_df


# #TODO 2024-03-02 12:12: - [ ] Could add weighted correlation if there is a dataframe for that and it's computed:
_df_raw_variable_names = ['simple_pf_pearson_merged_df', 'weighted_corr_merged_df']
_df_variables_names = ['laps_weighted_corr_merged_df', 'ripple_weighted_corr_merged_df', 'laps_simple_pf_pearson_merged_df', 'ripple_simple_pf_pearson_merged_df']

# # tbin_values_dict = {'laps': self.laps_decoding_time_bin_size, 'ripple': self.ripple_decoding_time_bin_size}
time_col_name_dict = {'laps': 'lap_start_t', 'ripple': 'ripple_start_t'} ## default should be 't_bin_center'

# fold older files:
# {'laps_marginals_df': 'lap_start_t', 'ripple_marginals_df': 'ripple_start_t', 'laps_time_bin_marginals_df':'t_bin_center', 'ripple_time_bin_marginals_df':'t_bin_center'}
    

# csv_sessions
# Extract each of the separate files from the sessions:

# final_sessions: Dict[types.session_str, Dict[str, Path]] = {}

final_sessions_loaded_laps_dict = {}
final_sessions_loaded_ripple_dict = {}
final_sessions_loaded_laps_time_bin_dict = {}
final_sessions_loaded_ripple_time_bin_dict = {}

final_sessions_loaded_simple_pearson_laps_dict = {}
final_sessions_loaded_simple_pearson_ripple_dict = {}

# stupid reudndant but compatible method
final_sessions_loaded_laps_wcorr_dict = {}
final_sessions_loaded_ripple_wcorr_dict = {}

final_sessions_loaded_laps_all_scores_dict = {}
final_sessions_loaded_ripple_all_scores_dict = {}


# final_sessions_loaded_nested_dict = {} # to be filled like
# {
#     'laps': {},
#     'ripple': {},
# }

final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (an_decoding_time_bin_size_str, a_path, an_export_datetime) in session_dict.items()}
                                                                                          for session_str, session_dict in csv_sessions.items()}

for session_str, session_dict in final_sessions.items():
    session_name = str(session_str)  # Extract session name from the filename
    if debug_print:
        print(f'processing session_name: {session_name}')
    curr_session_t_delta = t_delta_dict.get(session_name, {}).get('t_delta', None)
    if curr_session_t_delta is None:
        print(f'WARN: curr_session_t_delta is None for session_str = "{session_str}"')

    # Process each file type with its corresponding details
    process_and_load_file(session_dict, 'laps_marginals_df', final_sessions_loaded_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_marginals_df', final_sessions_loaded_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    process_and_load_file(session_dict, 'laps_time_bin_marginals_df', final_sessions_loaded_laps_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    process_and_load_file(session_dict, 'ripple_time_bin_marginals_df', final_sessions_loaded_ripple_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    process_and_load_file(session_dict, 'laps_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    process_and_load_file(session_dict, 'laps_weighted_corr_merged_df', final_sessions_loaded_laps_wcorr_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_weighted_corr_merged_df', final_sessions_loaded_ripple_wcorr_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    


    # process_and_load_file(session_dict, 'laps_all_scores_merged_df', final_sessions_loaded_laps_all_scores_dict, session_str, curr_session_t_delta, 'lap_start_t')
    process_and_load_file(session_dict, 'ripple_all_scores_merged_df', final_sessions_loaded_ripple_all_scores_dict, session_str, curr_session_t_delta, 'ripple_start_t')

    # prototype_flexible_process_and_load_file(session_dict, 'laps_marginals_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'lap_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'ripple_marginals_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'laps_time_bin_marginals_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 't_bin_center')
    # prototype_flexible_process_and_load_file(session_dict, 'ripple_time_bin_marginals_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 't_bin_center')
    # prototype_flexible_process_and_load_file(session_dict, 'laps_simple_pf_pearson_merged_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'lap_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'ripple_simple_pf_pearson_merged_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'laps_weighted_corr_merged_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'lap_start_t')
    # prototype_flexible_process_and_load_file(session_dict, 'ripple_weighted_corr_merged_df', final_sessions_loaded_nested_dict, session_str, curr_session_t_delta, 'ripple_start_t')


## Build across_sessions join dataframes:
all_sessions_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_dict.values()), axis='index', ignore_index=True)
# Add 'epoch_idx' column for compatibility:
all_sessions_laps_df['epoch_idx'] = all_sessions_laps_df['lap_idx']
all_sessions_ripple_df['epoch_idx'] = all_sessions_ripple_df['ripple_idx']

# *_time_bin marginals:
all_sessions_laps_time_bin_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_time_bin_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_time_bin_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_time_bin_dict.values()), axis='index', ignore_index=True)

# NEW ________________________________________________________________________________________________________________ #
all_sessions_simple_pearson_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_simple_pearson_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_simple_pearson_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_simple_pearson_ripple_dict.values()), axis='index', ignore_index=True)

if len(final_sessions_loaded_laps_wcorr_dict) > 0:
    all_sessions_wcorr_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_wcorr_dict.values()), axis='index', ignore_index=True)
else:
    all_sessions_wcorr_laps_df = None # empty df would be better

all_sessions_wcorr_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_wcorr_dict.values()), axis='index', ignore_index=True)

# `*_all_scores_*`: __________________________________________________________________________________________________ #
# all_sessions_all_score_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_all_scores_dict.values()), axis='index', ignore_index=True)
all_sessions_all_scores_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_all_scores_dict.values()), axis='index', ignore_index=True)


if 'time_bin_size' not in all_sessions_laps_df:
    print('Uh-oh! time_bin_size is missing! This must be old exports!')
    print(f'\tTry to determine the time_bin_size from the filenames: {csv_sessions}')
    ## manual correction UwU
    time_bin_size: float = 0.025
    print(f'WARNING! MANUAL OVERRIDE TIME BIN SIZE SET: time_bin_size = {time_bin_size}. Assigning to dataframes....')
    all_sessions_laps_df['time_bin_size'] = time_bin_size
    all_sessions_ripple_df['time_bin_size'] = time_bin_size
    all_sessions_laps_time_bin_df['time_bin_size'] = time_bin_size
    all_sessions_ripple_time_bin_df['time_bin_size'] = time_bin_size
    print(f'\tdone.')
else:
    # Filter rows based on column: 'time_bin_size'
    all_sessions_laps_df = all_sessions_laps_df[all_sessions_laps_df['time_bin_size'].notna()]
    all_sessions_ripple_df = all_sessions_ripple_df[all_sessions_ripple_df['time_bin_size'].notna()]
    all_sessions_laps_time_bin_df = all_sessions_laps_time_bin_df[all_sessions_laps_time_bin_df['time_bin_size'].notna()]
    all_sessions_ripple_time_bin_df = all_sessions_ripple_time_bin_df[all_sessions_ripple_time_bin_df['time_bin_size'].notna()]




all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)]
all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df)]

# all_sessions_all_score_laps_df, all_sessions_all_scores_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_all_score_laps_df, all_sessions_all_scores_ripple_df)]
# all_sessions_all_score_laps_df = _common_cleanup_operations(all_sessions_all_score_laps_df)
all_sessions_all_scores_ripple_df = _common_cleanup_operations(all_sessions_all_scores_ripple_df)

all_sessions_simple_pearson_laps_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_simple_pearson_laps_df, all_sessions_wcorr_laps_df, should_drop_directional_columns=False, start_t_idx_name='delta_aligned_start_t')
all_sessions_simple_pearson_ripple_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_ripple_df, should_drop_directional_columns=False, start_t_idx_name='ripple_start_t')

# all_sessions_laps_time_bin_df # 601845 rows × 9 column

collected_outputs_directory: /nfs/turbo/umms-kdiba/Data/Output/collected_outputs
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-4-10_12-58-3_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-6-12_16-53-46_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-4-09_16-40-54_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-2006-6-12_16-53-46_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-fet11-01_12-58-54_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-18_GL_t_split_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-4-09_17-29-30_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-11-03_12-3-25_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-6-09_1-22-43_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL

In [ ]:
all_sessions_new_laps_df # 931 rows × 25 columns

In [7]:
all_sessions_all_scores_ripple_df

,start,stop,label,duration,is_user_annotated_epoch,is_valid_epoch,P_Long_LR,score_long_LR,velocity_long_LR,intercept_long_LR,...,coverage_short_RL,jump_short_RL,longest_sequence_length_ratio_short_RL,direction_change_bin_ratio_short_RL,congruent_dir_bins_ratio_short_RL,total_congruent_direction_change_short_RL,ripple_start_t,session_name,time_bin_size,delta_aligned_start_t
0,1.489151,1.924423,0,0.435271,False,True,2.213249e-01,0.326623,-241.985542,193.101564,...,0.460317,0.005264,0.214286,0.692308,0.076923,0.000000,1.489151,kdiba_pin01_one_fet11-01_12-58-54,0.075,-2055.736797
1,3.490896,4.041767,1,0.550871,False,True,2.137675e-01,0.304100,-172.627832,138.660700,...,0.333333,0.004480,0.111111,0.823529,0.176471,0.000000,3.490896,kdiba_pin01_one_fet11-01_12-58-54,0.075,-2053.735052
2,12.982910,13.469085,3,0.486174,False,True,2.422670e-01,0.211610,-446.160214,231.162742,...,0.380952,0.005152,0.187500,0.600000,0.400000,603.792353,12.982910,kdiba_pin01_one_fet11-01_12-58-54,0.075,-2044.243038
3,46.106580,46.596564,12,0.489984,False,True,1.060163e-01,0.387626,-21.313506,26.510865,...,0.317460,0.005264,0.187500,0.666667,0.400000,637.336372,46.106580,kdiba_pin01_one_fet11-01_12-58-54,0.075,-2011.119369
4,53.350137,53.520510,14,0.170373,False,True,7.041159e-03,0.538458,696.540775,130.657869,...,0.190476,0.005152,0.200000,1.000000,0.500000,238.535251,53.350137,kdiba_pin01_one_fet11-01_12-58-54,0.075,-2003.875811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3861,2034.655153,2034.765468,629,0.110315,False,True,1.134218e-24,0.952708,-627.288793,223.890681,...,0.161290,0.000683,0.333333,1.000000,0.500000,22.758138,2034.655153,kdiba_gor01_one_2006-6-08_14-26-15,0.075,823.097073
3862,2043.892191,2044.079430,630,0.187238,False,True,3.118945e-01,0.881103,-109.924817,29.667026,...,0.209677,0.003074,0.333333,0.800000,0.400000,155.513946,2043.892191,kdiba_gor01_one_2006-6-08_14-26-15,0.075,832.334111
3863,2045.095432,2045.393262,631,0.297830,True,True,1.361919e-01,0.513434,-77.201647,32.414214,...,0.322581,0.005464,0.200000,0.777778,0.666667,269.304639,2045.095432,kdiba_gor01_one_2006-6-08_14-26-15,0.075,833.537352
3864,2051.142292,2051.269043,632,0.126751,True,True,1.583111e-06,0.498821,2752.295872,-80.332283,...,0.290323,0.002618,0.500000,0.666667,0.333333,0.000000,2051.142292,kdiba_gor01_one_2006-6-08_14-26-15,0.075,839.584212


In [5]:
all_sessions_ripple_df # 17592 rows × 10 columns

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,session_name,delta_aligned_start_t,epoch_idx,time_bin_size
0,0.456004,0.543996,0.493835,0.506165,0,1.489151,kdiba_pin01_one_fet11-01_12-58-54,-2055.736797,0,0.025
1,0.573421,0.426579,0.631624,0.368376,1,3.490896,kdiba_pin01_one_fet11-01_12-58-54,-2053.735052,1,0.025
2,0.508765,0.491235,0.549421,0.450579,2,12.982910,kdiba_pin01_one_fet11-01_12-58-54,-2044.243038,2,0.025
3,0.547634,0.452366,0.521215,0.478785,3,46.106580,kdiba_pin01_one_fet11-01_12-58-54,-2011.119369,3,0.025
4,0.488264,0.511736,0.697056,0.302944,4,53.350137,kdiba_pin01_one_fet11-01_12-58-54,-2003.875811,4,0.025
...,...,...,...,...,...,...,...,...,...,...
4374,0.683844,0.316156,0.107841,0.892159,116,771.753988,kdiba_gor01_two_2006-6-12_16-53-46,300.686588,116,0.025
4375,0.670214,0.329786,0.490734,0.509266,117,779.004365,kdiba_gor01_two_2006-6-12_16-53-46,307.936965,117,0.025
4376,0.409451,0.590549,0.300195,0.699805,118,781.957078,kdiba_gor01_two_2006-6-12_16-53-46,310.889678,118,0.025
4377,0.858188,0.141812,0.150732,0.849268,119,782.336224,kdiba_gor01_two_2006-6-12_16-53-46,311.268824,119,0.025


In [ ]:
csv_files

In [9]:
# final_output_path = Path("../output/").resolve()
final_output_path = Path("output/").resolve()

# final_sessions
# {'kdiba_gor01_one_2006-6-08_14-26-15': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-17_0540PM-kdiba_gor01_one_2006-6-08_14-26-15-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-17_0540PM-kdiba_gor01_one_2006-6-08_14-26-15-(laps_marginals_df).csv')},
#  'kdiba_gor01_one_2006-6-09_1-22-43': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0838PM-kdiba_gor01_one_2006-6-09_1-22-43-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0838PM-kdiba_gor01_one_2006-6-09_1-22-43-(laps_marginals_df).csv')},
#  'kdiba_pin01_one_fet11-01_12-58-54': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv')}}

# Save out the four dataframes to CSVs:
final_dfs_dict = {"AcrossSession_Laps_per-Epoch": all_sessions_laps_df, "AcrossSession_Ripple_per-Epoch": all_sessions_ripple_df,
				   "AcrossSession_Laps_per-TimeBin": all_sessions_laps_time_bin_df, "AcrossSession_Ripple_per-TimeBin": all_sessions_ripple_time_bin_df,
				   "AcrossSession_NEW_Laps_per-Epoch": all_sessions_new_laps_df, "AcrossSession_NEW_Ripple_per-Epoch": all_sessions_new_ripple_df,
				   "AcrossSession_ALL_SCORES_Ripple_per-Epoch": all_sessions_all_scores_ripple_df,
				   }

final_csv_export_paths = {}
for a_name, a_final_df in final_dfs_dict.items():
	# save out one final DF to csv.
	out_csv_filename: str = f"{TODAY_DAY_DATE}_{a_name}.csv"
	a_final_csv_export_path = final_output_path.joinpath(out_csv_filename).resolve()
	a_final_df.to_csv(a_final_csv_export_path) # save to CSV.
	final_csv_export_paths[a_name] = a_final_csv_export_path
	
final_csv_export_paths


{'AcrossSession_Laps_per-Epoch': PosixPath('/home/halechr/repos/Spike3D/output/2024-03-14_GL_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': PosixPath('/home/halechr/repos/Spike3D/output/2024-03-14_GL_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': PosixPath('/home/halechr/repos/Spike3D/output/2024-03-14_GL_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': PosixPath('/home/halechr/repos/Spike3D/output/2024-03-14_GL_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_NEW_Laps_per-Epoch': PosixPath('/home/halechr/repos/Spike3D/output/2024-03-14_GL_AcrossSession_NEW_Laps_per-Epoch.csv'),
 'AcrossSession_NEW_Ripple_per-Epoch': PosixPath('/home/halechr/repos/Spike3D/output/2024-03-14_GL_AcrossSession_NEW_Ripple_per-Epoch.csv'),
 'AcrossSession_ALL_SCORES_Ripple_per-Epoch': PosixPath('/home/halechr/repos/Spike3D/output/2024-03-14_GL_AcrossSession_ALL_SCORES_Ripple_per-Epoch.csv')}

In [ ]:
all_sessions_new_laps_df, all_sessions_new_ripple_df

In [ ]:
all_sessions_new_ripple_df.columns





In [ ]:
all_sessions_laps_df

# 2024-03-02 - Get only the user-annotated ripples:

In [10]:
from pyphocorehelpers.indexing_helpers import partition_df

all_unique_session_names = all_sessions_new_ripple_df['session_name'].unique()
# all_unique_session_names

# all_sessions_new_ripple_df['is_user_annotated_epoch'].notna()
# all_sessions_user_annotated_laps_df = all_sessions_new_laps_df.dropna(axis='index', subset=['is_user_annotated_epoch'], inplace=False) # user doesn't annotate laps
all_sessions_user_annotated_ripple_df: pd.DataFrame = all_sessions_new_ripple_df.dropna(axis='index', subset=['is_user_annotated_epoch', 'is_valid_epoch'], inplace=False)
user_annotated_epoch_unique_session_names = all_sessions_user_annotated_ripple_df['session_name'].unique()
print(f'user_annotated_epoch_unique_session_names: {user_annotated_epoch_unique_session_names}')

unannotated_session_names = set(all_unique_session_names) - set(user_annotated_epoch_unique_session_names)
print(f'unannotated_session_names: {unannotated_session_names}')


_validity_partitioned_dfs = dict(zip(*partition_df(all_sessions_user_annotated_ripple_df, partitionColumn='is_valid_epoch')))
valid_ripple_df: pd.DataFrame = _validity_partitioned_dfs[True].drop(columns=['is_valid_epoch']).reset_index(drop=True)
invalid_ripple_df: pd.DataFrame = _validity_partitioned_dfs[False].drop(columns=['is_valid_epoch']).reset_index(drop=True)

_partitioned_dfs = dict(zip(*partition_df(valid_ripple_df, partitionColumn='is_user_annotated_epoch'))) # use `valid_ripple_df` instead of the original dataframe to only get those which are valid.
user_approved_ripple_df: pd.DataFrame = _partitioned_dfs[True].drop(columns=['is_user_annotated_epoch']).reset_index(drop=True)
user_rejected_ripple_df: pd.DataFrame = _partitioned_dfs[False].drop(columns=['is_user_annotated_epoch']).reset_index(drop=True)

## 2024-03-14 - 'is_valid_epoch' column
# 'is_valid_epoch'
user_approved_ripple_df


user_annotated_epoch_unique_session_names: ['kdiba_pin01_one_fet11-01_12-58-54' 'kdiba_pin01_one_11-02_19-28-0'
 'kdiba_pin01_one_11-02_17-46-44' 'kdiba_gor01_two_2006-6-09_22-24-40'
 'kdiba_gor01_two_2006-6-08_21-16-25' 'kdiba_gor01_two_2006-6-07_16-40-19'
 'kdiba_gor01_one_2006-6-12_15-55-31' 'kdiba_gor01_one_2006-6-09_1-22-43'
 'kdiba_gor01_one_2006-6-08_14-26-15']
unannotated_session_names: {'kdiba_vvp01_one_2006-4-09_17-29-30', 'kdiba_vvp01_two_2006-4-09_16-40-54', 'kdiba_vvp01_two_2006-4-10_12-58-3', 'kdiba_vvp01_one_2006-4-10_12-25-50', 'kdiba_pin01_one_11-03_12-3-25'}


,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,...,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff
0,1.042215e-02,9.895778e-01,0.956882,0.043118,10,72.773154,9.972771e-03,9.469092e-01,4.493823e-04,4.266861e-02,...,NaN,NaN,0.800542,0.293113,0.895772,-0.194961,0.800542,0.895772,-0.095230,NaN
1,3.545687e-01,6.454313e-01,0.788366,0.211634,21,127.406680,2.795298e-01,5.088360e-01,7.503882e-02,1.365953e-01,...,0.600822,0.556009,0.238433,0.333013,0.051031,-0.076972,0.238433,0.051031,0.187401,0.044812
2,3.116855e-01,6.883145e-01,0.933157,0.066843,23,135.956513,2.908515e-01,6.423056e-01,2.083395e-02,4.600891e-02,...,NaN,NaN,0.254018,0.052917,0.192099,-0.050706,0.254018,0.192099,0.061920,NaN
3,2.791767e-01,7.208233e-01,0.615443,0.384557,42,248.042369,1.718174e-01,4.436259e-01,1.073593e-01,2.771974e-01,...,0.502127,0.498228,0.172825,0.158592,0.076586,0.134138,0.172825,0.076586,0.096239,0.003899
4,4.703330e-01,5.296670e-01,0.592863,0.407137,58,403.414434,2.788429e-01,3.140199e-01,1.914901e-01,2.156471e-01,...,NaN,NaN,-0.190983,-0.190365,-0.125331,-0.134426,-0.190983,-0.125331,0.065652,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,9.999689e-01,3.114813e-05,0.195342,0.804658,587,1925.050120,1.953362e-01,6.084548e-06,8.046326e-01,2.506358e-05,...,NaN,NaN,0.889109,0.893190,0.941818,0.859974,0.893190,0.859974,0.033216,NaN
411,5.294251e-01,4.705749e-01,0.264702,0.735298,592,1966.702763,1.401401e-01,1.245623e-01,3.892850e-01,3.460126e-01,...,-0.091480,-0.290040,-0.147887,-0.070073,-0.032639,-0.030354,-0.070073,-0.030354,0.039719,-0.198560
412,2.850322e-16,1.000000e+00,0.952789,0.047211,599,1998.452665,2.715756e-16,9.527890e-01,1.345665e-17,4.721097e-02,...,0.566058,0.594397,0.244235,-0.938981,0.440084,-0.834902,0.244235,0.440084,-0.195849,-0.028340
413,6.770086e-01,3.229914e-01,0.201167,0.798833,608,2045.095432,1.361919e-01,6.497528e-02,5.408167e-01,2.580161e-01,...,NaN,NaN,-0.045736,-0.445990,-0.130133,-0.365178,-0.445990,-0.365178,0.080812,NaN


In [ ]:
## want to compare difference between user approved and user rejected ripples

user_approved_ripple_df
user_rejected_ripple_df
# OUTPUTS: all_sessions_user_annotated_ripple_df, user_approved_ripple_df, user_rejected_ripple_df

In [ ]:
## Want to test for differences between long/short among only user approved ripples:

0.025


In [ ]:
df: pd.DataFrame = deepcopy(all_sessions_user_annotated_ripple_df)

### 2024-02-29 - 4pm - Filter the events for those meeting wcorr criteria:


In [ ]:
# df: pd.DataFrame = deepcopy(all_sessions_user_annotated_ripple_df)
df: pd.DataFrame = deepcopy(valid_ripple_df) # valid epochs, but not just those that the user approved
# df: pd.DataFrame = deepcopy(user_approved_ripple_df)

## INPUTS: df

min_wcorr_threshold: float = 0.33
min_wcorr_diff_threshold: float = 0.2

is_included_large_wcorr_diff = np.any((df[['wcorr_abs_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_high_wcorr = np.any((df[['long_best_wcorr', 'short_best_wcorr']].abs() > min_wcorr_threshold), axis=1)

df = df[is_included_high_wcorr]
df

# delta_aligned_start_t

In [ ]:
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from PIL import Image

from pyphocorehelpers.programming_helpers import copy_image_to_clipboard
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# fig_to_clipboard(fig)

# ## Test collapsed histograms-only results:
# histograms_only_all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
# 														#    enabled_time_bin_sizes=[0.03, 0.10],
# 															# enabled_time_bin_sizes=[0.03, 0.058, 0.10], # [0.03 , 0.044, 0.058, 0.072, 0.086, 0.1]
# 														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
# 														   main_plot_mode='default',
# 														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=False, figure_save_extension=['.html','.png'])
# histograms_only_fig_time_bin_laps, histograms_only_fig_time_bin_ripples = histograms_only_all_time_bin_session_figures[0]
# # histograms_only_fig_time_bin_laps.show()
# histograms_only_fig_time_bin_ripples.show()

all_sessions_user_annotated_ripple_df

In [ ]:
concatenated_ripple_df = deepcopy(all_sessions_user_annotated_ripple_df).infer_objects()
ripple_title_prefix = "User Annotated Sessions Ripples"

concatenated_ripple_df = deepcopy(user_approved_ripple_df).infer_objects()
ripple_title_prefix = "User Selected Ripples Only"

# concatenated_ripple_df = deepcopy(user_rejected_ripple_df).infer_objects()
# ripple_title_prefix = "User Rejected Ripples Only"

# concatenated_ripple_df = deepcopy(df)


# Create a bubble chart for ripples
enabled_time_bin_sizes = None
ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
fig_ripples = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=25, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=enabled_time_bin_sizes, main_plot_mode='default', title=ripple_title)
fig_ripples.show()

# fig_to_clipboard(fig_ripples)

In [ ]:
histogram_bins = 25
# concatenated_ripple_df = deepcopy(df)


# ripple_title_prefix = 'test'
# # Create a bubble chart for ripples
# ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
# # ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
# ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
# ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
# fig_ripples = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=histogram_bins, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=None,
#                                     main_plot_mode='separate_row_per_session', title=ripple_title)

concatenated_ripple_df = deepcopy(all_sessions_user_annotated_ripple_df).infer_objects()
ripple_title_prefix = "User Annotated Sessions Ripples"

scatter_title = 'Sevveral Sessions'
# variable_name = 'P_Long'
variable_name = 'wcorr_abs_diff'
# variable_name = 'pearsonr_abs_diff'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}} # , 'color': 'time_bin_size'
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch", histnorm='probability density')
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples



In [ ]:
type(new_fig_ripples) # plotly.graph_objs._figure.Figure

from plotly.graph_objects import Figure as PlotlyFigure

isinstance(new_fig_ripples, PlotlyFigure)

In [ ]:

## User non-selected:
scatter_title = f'user_approved_ripple_df Sevveral Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_approved_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples

In [ ]:
# IDEA: The ones with clear replays (diagonal sequences in the decoded posteriors) are by definiition ambiguous, because there's not much difference between the long/short decoders.


In [ ]:
## User non-selected:
scatter_title = f'Non-selected Sevveral Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_rejected_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples


In [ ]:
# Laps test
concatenated_ripple_df = deepcopy(all_sessions_new_laps_df)

scatter_title = 'Sevveral Sessions'
variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} 
new_fig_ripples = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples


# Plotting functions

In [ ]:
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import PlottingHelpers
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results

# Example usage:
all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'])
fig_laps, fig_ripples = all_session_figures[0]


# fig_laps.show()
fig_ripples.show()
# fig_laps.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_laps.html")
# fig_ripples.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_ripples.html")

In [ ]:
fig_to_clipboard(fig_ripples)

In [ ]:
## time_bin version:
all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='separate_row_per_session',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=True, figure_save_extension=['.html','.png'])
fig_time_bin_laps, fig_time_bin_ripples = all_time_bin_session_figures[0]
# fig_time_bin_laps.show()
fig_time_bin_ripples.show()

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results, _helper_build_figure

## Test collapsed histograms-only results:
histograms_only_all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
															# enabled_time_bin_sizes=[0.03, 0.058, 0.10], # [0.03 , 0.044, 0.058, 0.072, 0.086, 0.1]
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='default',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=False, figure_save_extension=['.html','.png'])
histograms_only_fig_time_bin_laps, histograms_only_fig_time_bin_ripples = histograms_only_all_time_bin_session_figures[0]
# histograms_only_fig_time_bin_laps.show()
histograms_only_fig_time_bin_ripples.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plotly_pre_post_delta_scatter
# data_results_df: pd.DataFrame = deepcopy(all_sessions_laps_df) # all_sessions_laps_time_bin_df
# histogram_bins = 'auto'
histogram_bins: int = 25

new_laps_fig = plotly_pre_post_delta_scatter(data_results_df=deepcopy(all_sessions_laps_df), out_scatter_fig=fig_laps, histogram_bins=histogram_bins)
new_laps_fig

# Dash-based Interactivity

In [ ]:
from typing import List, Dict, Optional, Tuple, Union
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
from dash.dash_table import DataTable, FormatTemplate
from dash.dash_table.Format import Format, Padding

import dash_bootstrap_components as dbc
import pandas as pd
from pathlib import Path
# import plotly.express as px
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results, _helper_build_figure
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_blue_yellow_points
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _build_dash_app

# Incorporate data
# final_csv_export_paths = {'AcrossSession_Laps_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-30_AcrossSession_Laps_per-Epoch.csv'),
#  'AcrossSession_Ripple_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-30_AcrossSession_Ripple_per-Epoch.csv'),
#  'AcrossSession_Laps_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-30_AcrossSession_Laps_per-TimeBin.csv'),
#  'AcrossSession_Ripple_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-30_AcrossSession_Ripple_per-TimeBin.csv')}

final_csv_export_paths = {'AcrossSession_Laps_per-Epoch': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_NEW_Laps_per-Epoch': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_NEW_Laps_per-Epoch.csv'),
 'AcrossSession_NEW_Ripple_per-Epoch': Path('/home/halechr/repos/Spike3D/output/2024-03-01_AcrossSession_NEW_Ripple_per-Epoch.csv')}

final_dfs_dict = {a_name:pd.read_csv(a_path.resolve()) for a_name, a_path in final_csv_export_paths.items()}
app = _build_dash_app(final_dfs_dict, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end)

# Run the app
if __name__ == '__main__':
    print("Starting the app.")  # Add this line to check if it's executed
    app.run(port="8054", debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter)

In [ ]:
a_df = deepcopy(final_dfs_dict['AcrossSession_Laps_per-TimeBin'])
a_df
# print(list(a_df.columns))
# ['Unnamed: 0.1', 'Unnamed: 0', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'epoch_idx', 't_bin_center', 'session_name', 'delta_aligned_start_t', 'time_bin_size']

yellow_blue_relevent_columns = ['epoch_idx', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'P_Long', 'P_Short', 'P_LR', 'P_RL']
a_df = a_df[yellow_blue_relevent_columns]

# filter by time_bin_size first:
a_single_selected_time_bin_size: float = 0.1 # time_bin_size
a_df = a_df[a_df.time_bin_size.astype(float) == a_single_selected_time_bin_size]

## Filter by selected epoch index:
selected_epoch_idxs = [5, 6, 9]
a_df = a_df[a_df.epoch_idx.isin(selected_epoch_idxs)]

# filter by selected epoch index:
a_single_epoch_row_idx: int = 0
a_single_epoch_idx = selected_epoch_idxs[a_single_epoch_row_idx]
a_single_epoch_df = a_df[a_df.epoch_idx == a_single_epoch_idx]
a_single_epoch_df

## filter by session:
a_single_session_name: str = 'kdiba_vvp01_one_2006-4-10_12-25-50'
a_single_epoch_df = a_single_epoch_df[a_single_epoch_df.session_name == a_single_session_name]
a_single_epoch_df


In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plotly_plot_1D_most_likely_position_comparsions
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_blue_yellow_points

# def plot_blue_yellow(a_df, selected_epoch_idxs, a_single_session_name: str):

specific_point_list = [{'session_name': 'kdiba_vvp01_one_2006-4-10_12-25-50', 'time_bin_size': 0.03, 'epoch_idx': 0, 'delta_aligned_start_t': -713.908702568122}]

# def plot_blue_yellow(a_df, selected_epoch_idxs, a_single_session_name: str):

## Takes: a_df, selected_epoch_idxs, a_single_session_name
a_df
a_single_session_name: str = 'kdiba_vvp01_one_2006-4-10_12-25-50' # #TODO 2024-01-30 16:01: - [ ] Hardcoded session:
selected_epoch_idxs = [5, 6, 9]
fig = plot_blue_yellow_points(a_df, selected_epoch_idxs=selected_epoch_idxs, a_single_session_name=a_single_session_name)
fig.show()


# Matplotlib-based versions:

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_histograms

matplotlib.use('Qt5Agg')

In [ ]:
# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:'long_best_dir_simple_pearsonr'
_out0: "MatplotlibRenderPlots" = plot_histograms(all_sessions_laps_time_bin_df, 'Laps', 'All Sessions', "75 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms(all_sessions_ripple_time_bin_df, 'Ripples', 'All Sessions', "75 ms")

In [ ]:


fig_to_clipboard(_out0.figures[0])

In [ ]:
## all_sessions_new_laps_df, all_sessions_new_ripple_df
plot_histograms(all_sessions_new_laps_df, 'New Laps', 'All Sessions', "25 ms")
plot_histograms(all_sessions_new_ripple_df, 'New Ripples', 'All Sessions', "25 ms")


In [ ]:
all_sessions_new_laps_df

In [ ]:
len(all_sessions_new_laps_df['session_name'].unique()) # 10 sessions



In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:
num_unique_sessions: int = all_sessions_laps_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_laps_time_bin_df.time_bin_size.nunique(dropna=True)
plot_stacked_histograms(all_sessions_laps_time_bin_df, 'Laps', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes")

num_unique_sessions: int = all_sessions_ripple_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_ripple_time_bin_df.time_bin_size.nunique(dropna=True)
plot_stacked_histograms(all_sessions_ripple_time_bin_df, 'Ripples', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes")

## 2024-02-20 - New Pearsonr analysis

In [ ]:
new_column_names = ['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr']


def _add_best_direction_columns(_temp_all_sessions_new_ripple_df):
    """ Get the most-likely direction to determine which correlation to use
    
    """
    ## Get the most-likely direction to determine which correlation to use
    # _temp_all_sessions_new_ripple_df['best_dir_simple_pearsonr'] = pd.NA

    # _temp_all_sessions_new_ripple_df[_temp_all_sessions_new_ripple_df['P_LR'] > 0.5] # LR-likely plots

    ## Adds two new columns using the best direction as determined by the decoder: ['long_best_dir_simple_pearsonr', 'short_best_dir_simple_pearsonr']
    # Using numpy where
    _temp_all_sessions_new_ripple_df['long_best_dir_simple_pearsonr'] = np.where((_temp_all_sessions_new_ripple_df['P_LR'] > 0.5), _temp_all_sessions_new_ripple_df['long_LR_pf_peak_x_pearsonr'], _temp_all_sessions_new_ripple_df['long_RL_pf_peak_x_pearsonr'])
    _temp_all_sessions_new_ripple_df['short_best_dir_simple_pearsonr'] = np.where((_temp_all_sessions_new_ripple_df['P_LR'] > 0.5), _temp_all_sessions_new_ripple_df['short_LR_pf_peak_x_pearsonr'], _temp_all_sessions_new_ripple_df['short_RL_pf_peak_x_pearsonr'])
    ## Define a difference of the absolutes
    _temp_all_sessions_new_ripple_df['long_short_diff_best_dir_simple_pearsonr'] = (_temp_all_sessions_new_ripple_df['long_best_dir_simple_pearsonr'].abs() - _temp_all_sessions_new_ripple_df['short_best_dir_simple_pearsonr'].abs())
    return _temp_all_sessions_new_ripple_df


## Drop the dang NaN columns:
_temp_all_sessions_new_ripple_df= _add_best_direction_columns(all_sessions_new_ripple_df.dropna(subset=new_column_names))
_temp_all_sessions_new_laps_df= _add_best_direction_columns(all_sessions_new_laps_df.dropna(subset=new_column_names))
_temp_all_sessions_new_laps_df

In [ ]:
['long_best_dir_simple_pearsonr_abs']

plot_histograms(_temp_all_sessions_new_laps_df, 'Laps', 'Some Sessions', "25 ms", column='long_best_dir_simple_pearsonr')

In [ ]:
## Get only high-confidence replays:
_temp_high_confidence_ripple_df = _temp_all_sessions_new_ripple_df[_temp_all_sessions_new_ripple_df['P_Long'] > 0.85]
# plot_histograms(_temp_high_confidence_ripple_df, 'Ripple', 'Some Sessions', "25 ms", column='long_best_dir_simple_pearsonr')
plot_histograms(_temp_high_confidence_ripple_df, 'Ripple', 'Some Sessions', "25 ms", column='long_best_dir_simple_pearsonr')



In [ ]:
# _temp_all_sessions_new_ripple_df

plt.scatter(x='delta_aligned_start_t', y='long_best_dir_simple_pearsonr', data=_temp_all_sessions_new_ripple_df)
plt.show()

In [ ]:
# plt.scatter(x='delta_aligned_start_t', y='long_best_dir_simple_pearsonr', data=_temp_all_sessions_new_ripple_df)

# pile
plt.figure()
plt.ion()
# plt.scatter(x=_temp_all_sessions_new_ripple_df['delta_aligned_start_t'], y=_temp_all_sessions_new_ripple_df['long_best_dir_simple_pearsonr'])
plt.scatter(x=_temp_all_sessions_new_ripple_df['delta_aligned_start_t'], y=_temp_all_sessions_new_ripple_df['long_short_diff_best_dir_simple_pearsonr'])
plt.show()


In [ ]:
# plot_histograms(_temp_all_sessions_new_ripple_df, 'Ripple', 'Some Sessions', "25 ms", column='long_short_diff_best_dir_simple_pearsonr')





In [ ]:
import scipy.stats as stats

data_results_df: pd.DataFrame = deepcopy(_temp_all_sessions_new_ripple_df)
column = 'long_short_diff_best_dir_simple_pearsonr' # REJECT (t-statistic: 2.0498849677801947, p-value: 0.04048207344769608)
# column = 'short_best_dir_simple_pearsonr' # FAIL
# column = 'long_best_dir_simple_pearsonr' # FAIL (t-statistic: 0.5238142457664882, p-value: 0.600455893577746)

# get the pre-delta epochs
pre_delta_df = data_results_df[data_results_df['delta_aligned_start_t'] <= 0]
post_delta_df = data_results_df[data_results_df['delta_aligned_start_t'] > 0]

# Perform the t-test for two independent samples
# T_result = stats.ttest_ind(pre_delta_df[column].to_numpy(), post_delta_df[column].to_numpy(), equal_var=False)
T_result = stats.ttest_ind(pre_delta_df[column].to_numpy(), post_delta_df[column].to_numpy(), equal_var=False) # Welch's t-test
t_statistic, p_value = T_result

print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Decision making
alpha = 0.05  # commonly used significance level
if p_value < alpha:
    print("Reject the null hypothesis. There is a significant difference between the two group means.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the two group means.")


In [ ]:

all_sessions_new_ripple_df


In [ ]:
plt.show()

In [ ]:
time_bin_sizes: int = all_sessions_laps_time_bin_df['time_bin_size'].unique()
time_bin_sizes

In [ ]:
all_sessions_laps_time_bin_df.hist(column='P_Long')
plt.show()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import plot_all_epoch_bins_marginal_predictions
from attrs import define, field, Factory
from neuropy.utils.result_context import IdentifyingContext
matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# Switch to the desired interactivity mode
plt.interactive(True)


@define(slots=False)
class TwoCSV:
	""" simple class wrapper to emulate the object that holds the other dfs """
	laps_all_epoch_bins_marginals_df = field()
	ripple_all_epoch_bins_marginals_df = field()

save_figure = True
def _perform_write_to_file_callback(final_context, fig):
	print(f'final_context: {final_context}')
	# if save_figure:
	# 	fig.save_fig(
	# 	# return owning_pipeline_reference.output_figure(final_context, fig)
	# else:
	# 	pass # do nothing, don't save
	
# all_sessions_laps_time_bin_df

collector = plot_all_epoch_bins_marginal_predictions(TwoCSV(laps_all_epoch_bins_marginals_df=all_sessions_laps_df, ripple_all_epoch_bins_marginals_df=all_sessions_ripple_df), t_start=None, t_split=0.0, t_end=None,
													active_context=IdentifyingContext(), perform_write_to_file_callback=_perform_write_to_file_callback)
collector


result = tuple(collector.figures)
result[0]

In [ ]:
result[0].show()

In [ ]:
result[1]

In [ ]:
all_sessions_laps_df, all_sessions_ripple_df

# BAD 2024-02-08 - Temporal Timecourse of Long/Short Replays after Delta using cross-correlations and auto-correlations

In [ ]:
from pyphocorehelpers.indexing_helpers import partition_df

def _split_is_post_delta(df):
    df['is_post_delta'] = (df['delta_aligned_start_t'] > 0.0)
    return partition_df(df, partitionColumn='is_post_delta')[-1] # [-1] gets the actual dataframe list

def _split_long_likely(df):
    df['is_long_likely'] = (df['P_Long'] > 0.5)
    return partition_df(df, partitionColumn='is_long_likely')[-1] # [-1] gets the actual dataframe list

def _split_on_pre_post_delta_and_likely_track(df):
    pre_delta_ripple_df, post_delta_ripple_df = _split_is_post_delta(df)
    pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df = _split_long_likely(pre_delta_ripple_df)
    post_delta_long_likely_ripple_df, post_delta_short_likely_ripple_df = _split_long_likely(post_delta_ripple_df)
    return (pre_delta_ripple_df, post_delta_ripple_df), (pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df), (post_delta_long_likely_ripple_df, post_delta_short_likely_ripple_df)


# only allow one time bin size
# active_time_bin_size = 0.10

# active_time_bin_size = 0.025
active_time_bin_size = 0.03

## Get only those after the delta, which occurs at t=0 relative to delta_aligned_start_t:
filtered_all_sessions_laps_df = all_sessions_laps_df[all_sessions_laps_df['time_bin_size'] == active_time_bin_size].copy()
filtered_all_sessions_laps_df['is_long_likely'] = (filtered_all_sessions_laps_df['P_Long'] > 0.5)
(pre_delta_laps_df, post_delta_laps_df), (pre_delta_long_likely_laps_df, pre_delta_short_likely_laps_df), (post_delta_long_likely_laps_df, post_delta_short_likely_laps_df) = _split_on_pre_post_delta_and_likely_track(filtered_all_sessions_laps_df)

filtered_all_sessions_ripple_df = all_sessions_ripple_df[all_sessions_ripple_df['time_bin_size'] == active_time_bin_size].copy().drop(columns=['time_bin_size'])
filtered_all_sessions_ripple_df['is_long_likely'] = (filtered_all_sessions_ripple_df['P_Long'] > 0.5)
(pre_delta_ripple_df, post_delta_ripple_df), (pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df), (post_delta_long_likely_ripple_df, post_delta_short_likely_ripple_df) = _split_on_pre_post_delta_and_likely_track(filtered_all_sessions_ripple_df)


In [ ]:
laps_df_dict = {
	"all_post_delta_laps": filtered_all_sessions_laps_df,
	"long_likely_laps_df": long_likely_laps_df,
	"short_likely_laps_df": short_likely_laps_df,
}
	
ripples_df_dict = {
	"all_post_delta_ripple": filtered_all_sessions_ripple_df,
	"long_likely_ripple_df": long_likely_ripple_df,
	"short_likely_ripple_df": short_likely_ripple_df,
}

all_sessions_laps_df, all_sessions_ripple_df
all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df
earliest_delta_aligned_t_start, latest_delta_aligned_t_end

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
# import proplot as pplt
# %matplotlib inline
# %matplotlib auto
%matplotlib qt


In [ ]:
def plot_laps(laps_df_dict):
    """
    Consolidates the laps plots into a single figure
    Args:
        laps_df_dict (dict): A dictionary with keys as plot titles and values as DataFrame with 'delta_aligned_start_t' column
    """
    fig, axs = plt.subplots(3, figsize=(10, 15))
    fig.suptitle('Laps Plots')
    
    for i, (title, df) in enumerate(laps_df_dict.items()):
        pd.plotting.autocorrelation_plot(df.delta_aligned_start_t, ax=axs[i])
        axs[i].set_title(title)
        axs[i].set_xlabel('Lag (seconds)')
        
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

def plot_ripples(ripples_df_dict):
    """
    Consolidates the ripple plots into a single figure
    Args:
        ripples_df_dict (dict): A dictionary with keys as plot titles and values as DataFrame with 'delta_aligned_start_t' column
    """
    fig, axs = plt.subplots(3, figsize=(10, 15))
    fig.suptitle('Ripples Plots')
    
    for i, (title, df) in enumerate(ripples_df_dict.items()):
        pd.plotting.autocorrelation_plot(df.delta_aligned_start_t, ax=axs[i])
        axs[i].set_title(title)
        axs[i].set_xlabel('Lag (seconds)')
        
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

## BODY
def _plot_all_signal_correlations(filtered_all_sessions_laps_df, long_likely_laps_df, short_likely_laps_df, filtered_all_sessions_ripple_df, long_likely_ripple_df, short_likely_ripple_df):
    laps_df_dict = {
        "all_post_delta_laps": filtered_all_sessions_laps_df,
        "long_likely_laps_df": long_likely_laps_df,
        "short_likely_laps_df": short_likely_laps_df,
    }
    plot_laps(laps_df_dict)
        
    ripples_df_dict = {
        "all_post_delta_ripple": filtered_all_sessions_ripple_df,
        "long_likely_ripple_df": long_likely_ripple_df,
        "short_likely_ripple_df": short_likely_ripple_df,
    }
    plot_ripples(ripples_df_dict)



## Inputs: pre_delta_long_likely_laps_df, pre_delta_short_likely_laps_df, pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df
# long_likely_laps_df = pre_delta_long_likely_laps_df.copy()
# short_likely_laps_df = pre_delta_short_likely_laps_df.copy()

# long_likely_ripple_df = pre_delta_long_likely_ripple_df.copy()
# short_likely_ripple_df = pre_delta_short_likely_ripple_df.copy()

## Inputs: pre_delta_long_likely_laps_df, pre_delta_short_likely_laps_df, pre_delta_long_likely_ripple_df, pre_delta_short_likely_ripple_df

# _plot_all_signal_correlations(filtered_all_sessions_laps_df, long_likely_laps_df, short_likely_laps_df, filtered_all_sessions_ripple_df, long_likely_ripple_df, short_likely_ripple_df)
# plt.suptitle('Pre-$\Delta$ Time Correlations')

# _plot_all_signal_correlations(filtered_all_sessions_laps_df, pre_delta_long_likely_laps_df.copy(), pre_delta_short_likely_laps_df.copy(), filtered_all_sessions_ripple_df, pre_delta_long_likely_ripple_df.copy(), pre_delta_short_likely_ripple_df.copy())
_plot_all_signal_correlations(filtered_all_sessions_laps_df, post_delta_long_likely_laps_df.copy(), post_delta_short_likely_laps_df.copy(), filtered_all_sessions_ripple_df, post_delta_long_likely_ripple_df.copy(), post_delta_short_likely_ripple_df.copy())
plt.suptitle('Post-$\Delta$ Time Correlations - All Sessions')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# sns.kdeplot(filtered_all_sessions_ripple_df['delta_aligned_start_t'])
# sns.kdeplot(filtered_all_sessions_ripple_df['delta_aligned_start_t'])
# sns.kdeplot(filtered_all_sessions_ripple_df, x='delta_aligned_start_t', hue='is_post_delta', y='is_long_likely')
sns.kdeplot(filtered_all_sessions_ripple_df, x='delta_aligned_start_t', hue='is_long_likely')
plt.show()

In [ ]:
sns.kdeplot(long_likely_ripple_df, x='delta_aligned_start_t')
plt.show()

In [ ]:
sns.kdeplot(short_likely_ripple_df['delta_aligned_start_t'])
plt.show()

In [ ]:
sns.kdeplot(post_delta_long_likely_ripple_df['delta_aligned_start_t'])
plt.show()

In [ ]:
sns.kdeplot(post_delta_short_likely_ripple_df['delta_aligned_start_t'])

In [ ]:
# Assuming series1 and series2 are your two pandas Series
correlation = long_likely_ripple_df.delta_aligned_start_t.corr(short_likely_ripple_df.delta_aligned_start_t)
print('Correlation: ', correlation)

# # Scatter plot
# plt.scatter(long_likely_ripple_df.delta_aligned_start_t, short_likely_ripple_df.delta_aligned_start_t)
# plt.title('Scatter Plot')
# plt.xlabel('Series 1')
# plt.ylabel('Series 2')
# plt.show()

In [ ]:

 # delta_aligned_start_t

# filtered_all_sessions_laps_df.delta_aligned_start_t.autocorr()

# plot_acf(filtered_all_sessions_laps_df.delta_aligned_start_t)
plt.show()

In [ ]:
plt.close('all')

In [ ]:
import pycorrelate as pyc

# compute lags in timestamp units (not in seconds!)
# to avoid floating point inacuracies
# bins_per_dec = 10
# unit = np.timedelta64()

# bins = pyc.make_loglags(1, 8, bins_per_dec)[bins_per_dec//2:]
# Convert the float array to timedelta64 by multiplying with '1s'
# bins = (bins * 1e9).astype('timedelta64[ns]')

unit = 1e9
# convert_to_time_units = lambda t: (t * 1e9).astype('timedelta64[ns]')
convert_to_time_units = lambda t: (t) #.astype('timedelta64[ns]')

# compute lags in sec. then convert to timestamp units
# bins = pyc.make_loglags(-1, 5, 20)
# bins = pyc.make_loglags(-1, 5, 5)
bins = pyc.make_loglags(-1, 2, 5)

# Convert the float array to timedelta64 by multiplying with '1s'
# bins = (bins * 1e9).astype('timedelta64[ns]')
bins = convert_to_time_units(bins)

long_likely_epoch_times = convert_to_time_units(deepcopy(long_likely_laps_df['delta_aligned_start_t'].to_numpy()))
short_likely_epoch_times = convert_to_time_units(deepcopy(short_likely_laps_df['delta_aligned_start_t'].to_numpy()))

# long_likely_epoch_times = convert_to_time_units(deepcopy(long_likely_ripples_df['delta_aligned_start_t'].to_numpy()))
# short_likely_epoch_times = convert_to_time_units(deepcopy(short_likely_ripples_df['delta_aligned_start_t'].to_numpy()))

long_likely_autocorrelation = pyc.pcorrelate(long_likely_epoch_times, long_likely_epoch_times, bins)
short_likely_autocorrelation = pyc.pcorrelate(short_likely_epoch_times, short_likely_epoch_times, bins)
long_short_likely_cross_correlation = pyc.pcorrelate(long_likely_epoch_times, short_likely_epoch_times, bins)

long_likely_autocorrelation
short_likely_autocorrelation
long_short_likely_cross_correlation

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

# Convert data to DataFrame
t_bins = deepcopy(bins.astype('float'))  # convert timedelta64[ns] to float
t_bins = (t_bins[1:] + t_bins[:-1]) / 2

df1 = pd.DataFrame({'Bins': t_bins, 'Correlation': long_likely_autocorrelation, 'Type': 'Long Autocorrelation'})
df2 = pd.DataFrame({'Bins': t_bins, 'Correlation': short_likely_autocorrelation, 'Type': 'Short Autocorrelation'})
df3 = pd.DataFrame({'Bins': t_bins, 'Correlation': long_short_likely_cross_correlation, 'Type': 'Cross Correlation'})

df = pd.concat([df1, df2, df3])

# Plot using Plotly Express
fig = px.line(df, x="Bins", y="Correlation", color="Type", facet_row="Type", 
              labels={'Bins': 'Bins (seconds)', 'Correlation': 'Correlation'},
              title='Correlation vs Bins')
# fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

fig

In [ ]:
G = long_short_likely_cross_correlation.copy()
# bins_unit = bins*unit
bins_unit = bins.copy()
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(bins_unit, np.hstack((G[:1], G)), drawstyle='steps-pre')
plt.xlabel('Time (s)')
#for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
plt.grid(True); plt.grid(True, which='minor', lw=0.3)
plt.xscale('log')
# plt.xlim(30e-9, 2)
plt.title('Cross-correlation of Long and Short')

long_short_likely_cross_correlation

In [ ]:
# Tweak here matplotlib style
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.sans-serif'].insert(0, 'Arial')
mpl.rcParams['font.size'] = 10
# %config InlineBackend.figure_format = 'retina'
# %matplotlib qt
# matplotlib.use('Qt5Agg')

def plot_correlate(G):
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.plot(bins, np.hstack((G[:1], G)), drawstyle='steps-pre')
    plt.xlabel('Time (s)')
    #for x in bins[1:]: plt.axvline(x*unit, lw=0.2)  # to mark bins
    plt.grid(True); plt.grid(True, which='minor', lw=0.3)
    plt.xscale('log')
    plt.xlim(30e-9, 2)
    return fig, ax

# fig, ax = plot_correlate(long_likely_autocorrelation)
fig, ax = plot_correlate(long_short_likely_cross_correlation)
# fig.show()
fig